## Retrieval Augmented Generation (RAG)

### Instalacja bibliotek

In [6]:
!pip install langchain_text_splitters

### Budowa VectorStore (FAISS) and Retrievera

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

load_dotenv()

docs = [
    "LangChain this framework to pracy with LLM.",
    "RAG łączy retrieval kontekstu with generacją answers.",
    "FAISS this biblioteka to przechowywania and wyszukiwania embeddingów.",
    "Retriever służy to wyszukiwania najbardziej podobnych dokumentów to queries użytkownika. Retriever może zwrócić różną liczbę pasujących dokumentów określona in parametrze k. Retriever wykorzystuje różne algorytmy podobieństwa tekstów, np. dopasowanie kosinusowe, odległość euklidesowa, MMR."
]

splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
split = splitter.create_documents(docs)

print(f"Number of chunks: {len(split)}")

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

query = "After co używa się retrievera?"

context = retriever.invoke(query)
print("Znaleziony chunk:")
for i, c in enumerate(context, 1):
    print(f"{and}.", c.page_content)

Number of chunks: 7
Znaleziony fragment:
1. Retriever służy do wyszukiwania najbardziej podobnych dokumentów do zapytania użytkownika. Retriever
2. Retriever może zwrócić różną liczbę pasujących dokumentów określona w parametrze k. Retriever


### Prosty chain RAG (prompt + kontekst + LLM)

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "Udziel precyzyjnej answers wyłącznie on podstawie KONTEKSTU. If brak danych — powiedz, that nie wiesz."),
    ("system", "KONTEKST:\\n{context}"),
    ("user", "{question}")
])

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("Czym jest FAISS and to czego służy?"))

FAISS to biblioteka do przechowywania i wyszukiwania embeddingów. Służy do efektywnego wyszukiwania najbardziej podobnych dokumentów do zapytania użytkownika.


### Example RAG - cały program

In [9]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

#  Model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

#  Documents źródłowe
docs = [
    "LangChain this framework to pracy with LLM.",
    "RAG łączy dopasowanie kontekstu with generacją answers.",
    "FAISS this biblioteka to przechowywania and wyszukiwania embeddingów."
]

#  Split
splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
splits = splitter.create_documents(docs)

#  Embeddings + vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(splits, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

#  Prompt RAG
prompt = ChatPromptTemplate.from_messages([
    ("system", "Odpowiadaj tylko on podstawie kontekstu:\\n{context}"),
    ("user", "{question}")
])

#  Pipeline
rag_chain = (
    {
        "context": lambda x: retriever.invoke(x["question"]),
        "question": lambda x: x["question"]
    }
    | prompt
    | llm
    | StrOutputParser()
)


print(rag_chain.invoke({"question": "Co this jest FAISS?"}))

FAISS to biblioteka do przechowywania i przeszukiwania dużych zbiorów wektorów.


### RAG with pętlą and ewaluacją

In [10]:
from langchain_core.prompts import ChatPromptTemplate

eval_prompt = ChatPromptTemplate.from_messages([
    ("system", "Oceń answer."),
    ("user", "Question: {question}\\nOdpowiedź: {answer}\\nCzy answer jest poprawna? Odpowiedz yes/no.")
])

def rag_with_eval(question, max_retries):
    for attempt in range(max_retries):
        context = retriever.invoke(question)
        answer = (prompt | llm | StrOutputParser()).invoke({"context": context, "question": question})
        eval_result = (eval_prompt | llm | StrOutputParser()).invoke({"question": question, "answer": answer})
        print(f"Result ewaluacji {eval_result}")
        if "yes" in eval_result.lower():
            return f"✅ Answer zaakceptowana:\\n{answer}"
        print(f"❌ Answer: {answer}\\n odrzucona, ponawiam próbę...")
    return "Nie udało się uzyskać poprawnej answers."

print(rag_with_eval("Co this jest RAG?", max_retries=3))

Wynik ewaluacji Yes.
✅ Odpowiedź zaakceptowana:
RAG łączy dopasowanie kontekstu z generacją odpowiedzi.
